# Homework 14

In [17]:
import requests as req
import pandas as pd
from bs4 import BeautifulSoup

## Excercise 1

In [29]:
soup = BeautifulSoup(req.get('https://habr.com/ru/all/').text, 'html.parser')
KEYWORDS = ['моделирования', 'дайджест']

def make_post_df():

    links = []
    for post in soup.find_all('article', class_='post post_preview'):
        title = post.find('a', class_='post__title_link').text
        link = post.find('a', class_='post__title_link').get('href')
        text = post.find('div', class_='post__text').text

        for keyword in KEYWORDS: # search for text or title to contain desired keywords
            if keyword.lower() in title.lower().split( ) or keyword.lower() in text.lower().split( ):
                links.append(link)

    habr_posts_full = pd.DataFrame()
    for link in links:
        full_soup = BeautifulSoup(req.get(link).text, 'html.parser')
        title = full_soup.find('span', class_='post__title-text').text
        date = full_soup.find('span', class_='post__time').get('data-time_published')
        text = full_soup.find('div', class_='post__text').text

        row = {'date': pd.to_datetime(date), 'title': title, 'link': link, 'text': text}
        habr_posts_full = pd.concat([habr_posts_full, pd.DataFrame([row])])

    return habr_posts_full

make_post_df()

,date,title,link,text
0,2021-05-23 15:43:00+00:00,Дайджест интересных материалов для мобильного ...,https://habr.com/ru/company/productivity_insid...,"На этой неделе у нас новая Google I/O, доступн..."
0,2021-05-23 15:22:00+00:00,Как AutoML помогает создавать модели композитн...,https://habr.com/ru/company/spbifmo/blog/558450/,\nВ лаборатории моделирования природных систем...


## Excercise 2

In [19]:
import json
import time

In [20]:
def breach_json(list_=['xxx@yahoo.com']):

    URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
    headers_ = {'Host': 'identityprotection.avast.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
        'Accept-Encoding': 'gzip, deflate, br',
        'vaar-header-captcha-response-token': '03AGdBq277VhhdclU5YlLD90uZ185oEgGJVulv_QqyfW2yMP5CJ8KHs2bTfNc0JLnXOQDPK2JqBvYzCNgqC-yqAWIWkEqf2yDR_sdp8pxqsfq7xiC2jIOnRpDgCtmTFPPnPF4741S06CONYxDE44LQ5_lTjECgzqY87Si-NMH1ymJCrp3Lim9AKWhU0hnZgNWZX45d6RWKjQke5NjlISEMA4QzViGGaM-O-pggluEsj6JVPouCYZs0gzyqPXHuodJfaIZczs-PS5BUkqvPC1R7JTVUMVBbz8vJgrDvNDK2T-hPnslghOBWcF8t7_Wv6bKsmvX60h9oN0r4BThm7HHWG4NUny2_dpMmTxnUhYu3_lH9v-3jvu_4C-p64cy0XqFYgbLsnARVmRJq_f2rinDHOADQhL_xslIav5bZS9xqrB9WCoNcxN8FcHxBb4AThYH3Awe30YBF4Yaz',
        'Vaar-Header-App-Product': 'hackcheck-web-avast',
        'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
        'Vaar-Header-App-Build-Version': '1.0.0',
        'Vaar-Version': '0',
        'Content-Type': 'application/json;charset=utf-8',
        'Content-Length': '43',
        'Origin': 'https://www.avast.com',
        'Connection': 'keep-alive',
        'Referer': 'https://www.avast.com/'}

    breaches = []
    for email in list_:
        payload = json.dumps({"emailAddresses": [email]})
        check = req.post(URL, headers=headers_, data=payload)
        breaches.append(check.json())
        time.sleep(3)

    return breaches

breaches = breach_json()

In [21]:
def breach_df(response=breaches):

    breach_info_df = pd.DataFrame()
    for json_ in response:
        if json_ is not None:
            for info in json_['breaches'].values():
                row = {'date': pd.to_datetime(info['publishDate']), 'site': info['site'], 'description': info['description']}
                breach_info_df = pd.concat([breach_info_df, pd.DataFrame([row])])

    return breach_info_df

breach_df()

,date,site,description
0,2020-12-10 00:00:00+00:00,slyck.com,"In November 2020, a collection of over 23,000 ..."
0,2020-12-17 00:00:00+00:00,liceojuanramonjimenez.edu.co,"In November 2020, a collection of over 23,000 ..."
0,2020-12-10 00:00:00+00:00,bulapin.cn,"In November 2020, a collection of over 23,000 ..."
0,2020-12-10 00:00:00+00:00,licitatia.ro,"In November 2020, a collection of over 23,000 ..."
0,2020-05-14 00:00:00+00:00,minted.com,"In May 2020, the hacking group “ShinyHunters” ..."
...,...,...,...
0,2020-12-10 00:00:00+00:00,heraldicahispana.info,"In November 2020, a collection of over 23,000 ..."
0,2020-02-20 00:00:00+00:00,mysteryscience.com,"In January 2020, the online science education ..."
0,2020-12-10 00:00:00+00:00,rhodes-ranch.com,"In November 2020, a collection of over 23,000 ..."
0,2020-12-10 00:00:00+00:00,65.111.174.158,"In November 2020, a collection of over 23,000 ..."


## Excercise 3

In [31]:
#TOKEN =
GROUP = 'fide'

def get_last_posts(group=GROUP):

    params = {'access_token': TOKEN,
              'domain': group,
              'v': 5.21
              }

    r = req.get('https://api.vk.com/method/wall.get', params=params)

    vk_post_df = pd.DataFrame()
    for post in r.json()['response']['items']:
        row = {'date': pd.to_datetime(post['date'], unit='s'),'text': post['text']}
        vk_post_df = pd.concat([vk_post_df, pd.DataFrame([row])])

    return vk_post_df

get_last_posts()

,date,text
0,2021-05-21 22:12:48,Гран-при ФИДЕ среди женщин (FIDE Women's Grand...
0,2021-05-23 14:31:04,В эти минуты в Гибралтаре проходят партии втор...
0,2021-05-23 12:00:07,«Анатолию Евгеньевичу Карпову сегодня исполнил...
0,2021-05-23 10:56:24,23 мая в Гибралтаре проходит второй тур четвёр...
0,2021-05-23 06:40:00,Владимир Крамник поздравил Анатолия Карпова с ...
0,2021-05-23 05:30:00,23 мая день рождения отмечает двенадцатый чемп...
0,2021-05-22 19:30:00,Интервью после первого тура Гран-при ФИДЕ сред...
0,2021-05-22 19:10:00,Интервью после первого тура Гран-при ФИДЕ сред...
0,2021-05-22 18:54:11,Интервью после первого тура Гран-при ФИДЕ сред...
0,2021-05-22 18:46:44,Интервью после первого тура Гран-при ФИДЕ сред...
